In [1]:
'''
This code belongs to Miguel Pérez Cerdán. Should you have any comment or suggestion please feel free to contact
me through my email address: miguelpecerdan@gmail.com.

I will be updating the code periodically with new data and analysis. You can use it for your own studies as well.
'''

'\nThis code belongs to Miguel Pérez Cerdán. Should you have any comment or suggestion please feel free to contact\nme through my email address: miguelpecerdan@gmail.com.\n\nI will be updating the code periodically with new data and analysis. You can use it for your own studies as well.\n'

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm
import csv
import pathlib
import os
import warnings


# Constantes que vamos a usar durante todo el estudio.

In [3]:
# Estaciones según numeración del ayuntamiento
estaciones = [4, 8, 11, 16, 17, 18, 24, 27, 35, 36, 38, 39, 40, 47, 48,
              49, 50, 54, 55, 56, 57, 58, 59, 60]

# Nombres correspondientes a cada una de las estaciones
estaciones_nombres = ['Plaza España', 'Escuelas Aguirre',
                      'Av. Ramón y Cajal', 'Arturo Soria',
                      'Villaverde Alto', 'Calle Farolillo',
                      'Casa de Campo', 'Barajas', 'Pza. del Carmen',
                      'Moratalaz', 'Cuatro Caminos',
                      'Barrio del Pilar', 'Vallecas', 'Méndez Álvaro',
                      'Pº. Castellana', 'Retiro', 'Pza. Castilla',
                      'Ensanche Vallecas', 'Urb. Embajada (Barajas)',
                      'Plaza Elíptica', 'Sanchinarro', 'El Pardo',
                      'Parque Juan Carlos I', 'Tres Olivos']

# Diccionario de las estaciones con su código de distrito correspondiente
dict_estaciones = {'Plaza España': '09', 'Escuelas Aguirre': '04',
                   'Av. Ramón y Cajal': '05',
                   'Arturo Soria': '15', 'Villaverde Alto': '17',
                   'Calle Farolillo': '11', 'Casa de Campo': '09',
                   'Barajas': '21', 'Pza. del Carmen': '01',
                   'Moratalaz': '14', 'Cuatro Caminos': '07',
                   'Barrio del Pilar': '08', 'Vallecas': '13',
                   'Méndez Álvaro': '02', 'Pº. Castellana': '05',
                   'Retiro': '03', 'Pza. Castilla': '06',
                   'Ensanche Vallecas': '18',
                   'Urb. Embajada (Barajas)': '21',
                   'Plaza Elíptica': '12', 'Sanchinarro': '16',
                   'El Pardo': '08', 'Parque Juan Carlos I': '21',
                   'Tres Olivos': '08'}

# Coordenadas geográficas de las estaciones
coordenadas = {
    'Estacion': ['Plaza España', 'Escuelas Aguirre', 'Av. Ramón y Cajal',
                 'Arturo Soria', 'Villaverde Alto', 'Calle Farolillo',
                 'Casa de Campo', 'Barajas', 'Pza. del Carmen',
                 'Moratalaz',
                 'Cuatro Caminos', 'Barrio del Pilar', 'Vallecas',
                 'Méndez Álvaro', 'Pº. Castellana', 'Retiro',
                 'Pza. Castilla',
                 'Ensanche Vallecas', 'Urb. Embajada (Barajas)',
                 'Plaza Elíptica', 'Sanchinarro', 'El Pardo',
                 'Parque Juan Carlos I', 'Tres Olivos'],
    'Longitud': ['3°42\'43.91"O', '3°40\'56.22"O', '3°40\'38.5"O',
                 '3°38\'21.17"O', '3°42\'47.89"O', '3°43\'54.61"O',
                 '3°44\'50.44"O', '3°34\'48.10"O', '3°42\'11.4"O',
                 '3°38\'43.02"O', '3°42\'25.64"O', '3°41\'41.53"O',
                 '3°39\'5.5"O', '3°41\'12.57"O', '3°41\'25.34"O',
                 '3°40\'57.3"O', '3°41\'19.48"O', '3°36\'43.7"O',
                 '3°34\'50.03"O', '3°43\'7.54"O', '3°39\'37.86"O',
                 '3°46\'28.62"O', '3°36\'32.66"O', '3°41\'23.03"O'],
    'Latitud': ['40°25\'25.98"N', '40°25\'17.63"N', '40°27\'5.29"N',
                '40°26\'24.20"N', '40°20\'49.74"N', '40°23\'41.22"N',
                '40°25\'9.69"N', '40°28\'36.93"N', '40°25\'9.15"N',
                '40°24\'28.64"N', '40°26\'43.97"N', '40°28\'41.64"N',
                '40°23\'17.33"N', '40°23\'53.17"N', '40°26\'23.61"N',
                '40°24\'52"N', '40°27\'56.1"N', '40°22\'22.48"N',
                '40°27\'44.51"N', '40°23\'6.1"N', '40°29\'39.12"N',
                '40°31\'4.97"N', '40°27\'54.90"N', '40°30\'1.97"N']}

# Diccionario con los gases y sus claves en las tablas del Ayuntamiento
claves_gases = {1:'SO2', 6:'CO', 7:'NO', 8:'NO2', 9:'PM2.5', 10:'PM10', 12:'NOx', 14:'O3',
                20:'TOL', 30:'BEN', 35:'EBE', 37:'MXY', 38:'PXY', 39:'OXY', 42:'TCH', 43:'CH4'}


In [4]:
def limpieza(df, mes, year):
    
    '''
    Usamos este métódo para hacer una limpieza inicial de los datos que nos da el ayuntamiento. 
    Eleminamos las columnas que no nos hacen falta y filtramos por el gas en cuestión del que queremos hacer el 
    estudio.
    
    Entrada: 
        Dataframe con los datos en bruto del ayuntamiento.
        Mes a estudiar.
        Año.
        
    Salida:
        Dataframe con los datos ya limpios. 
        Mes.
        El código del gas utilizado.
        Año
    '''
    
    df.drop(['PROVINCIA', 'MUNICIPIO', 'PUNTO_MUESTREO', 'MES', 'ANO'], axis=1, inplace=True)

    for key in claves_gases.keys():
        
        df_gas = df[df.MAGNITUD == key]
        df_limpieza = df_gas.copy()
        df_limpieza = df_limpieza.reset_index()
        df_limpieza.drop(['ESTACION', 'MAGNITUD'], axis = 1, inplace=True)

        limpieza_nulos(df_limpieza, df_gas, mes, claves_gases[key], year)


In [5]:
def limpieza_nulos(df_limpieza, df_gas, mes, gas, year):
    
    '''
    En este método eliminaremos los datos que el Ayuntamiento ha considerado como NO válidos durante la toma de datos. 
    Estos vienen marcados con una N al lado de la casilla correspondiente al valor de la medición. 
    Los datos serán sustituidos por valores nulos. 
    
    Entrada: 
        Dataframe con los datos ya limpios. 
        Mes.
        El código del gas utilizado.
        Año.
    
    Salida: 
        Dataframe limpio, sin datos no válidos.
        Mes.
        Gas.
        Año.
    '''
    
    
    nombre = 'H01'
    for i in df_limpieza:
        for j, row_value in df_limpieza.iterrows():
            if row_value[i] == 'N':
                df_gas[nombre].iloc[j] = np.nan
        nombre = i
    df_dia = df_gas[['DIA', 'ESTACION', 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08',
                  'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17',
                  'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24']].copy()
    fill_rows(df_dia, mes, gas, year)
    

In [6]:
def fill_rows(df_dia, mes, gas, year):
    
    '''
    En este método ya generaremos el dataframe definitivo y lo guardaremos como un .CSV para poder tenerlo en futuros
    análisis. 
    
    Entrada: 
        Dataframe limpio, sin datos no válidos.
        Mes.
        Gas.
        Año.
        
    Salida:
        .CSV con los datos para futuros estudios. 
    '''
    
    if mes == 'ene' or 'mar' or 'may' or 'jul'or'ago'or 'oct'or'dic':
        dias = np.linspace(1,31,31, dtype = int)
    elif mes == 'feb':
        dias = np.linspace(1,28,28, dtype = int)
    else:
        dias = np.linspace(1,30,30, dtype = int)

    
    for j in estaciones:
        tabla_estacion = pd.DataFrame(columns=df_dia.columns)
        df_prueba = pd.DataFrame()
        df_prueba = df_dia[df_dia.ESTACION == j]  
        df_prueba.set_index('DIA', inplace=True)
        for k in dias-1:  
            
            try:
                if (k+1 not in df_prueba.index):   
                    tabla_estacion = tabla_estacion.append(pd.Series(), ignore_index=True)
                else:
                    tabla_estacion = tabla_estacion.append(df_prueba.iloc[k])
            except (IndexError):
                tabla_estacion = tabla_estacion.append(pd.Series(), ignore_index=True)
                
        tabla_estacion = tabla_estacion.reset_index()
        tabla_estacion.drop(['index'], axis = 1, inplace=True)
        
        if os.path.isdir('./Tablas/'):
            pass
        else:
            os.mkdir('./Tablas/')
            
        if os.path.isdir('./Tablas/year_%s/' %year):
            pass
        else:
            os.mkdir('./Tablas/year_%s/' %year)
        
        tabla_estacion.to_csv('./Tablas/year_%s/tabla_corregida_%s_%s_%s.csv'%(year, mes, gas, j), sep=';') 


In [7]:
'''
Punto inicial del programa. Pedirá por pantalla el año a estudiar (por ahora 2019 o 2020 hasta 08/2020) y cargará
los dataFrames de cada mes.
'''


warnings.filterwarnings("ignore")

year = input('Introduce el año a estudiar: \n2020 -> 20\n2019 -> 19\n\n\n') 

print('\n\nTenga en cuenta que no todas las estaciones de medición tienen la capacidad de medir todos los gases.\n'
        'Para más información, consultar: https://datos.madrid.es/portal/site/egob/ \n\n')


meses = ['ene', 'feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']

#Comprobamos cuantos meses tenemos en nuestra carpeta:
loc = './datos_ayuntamiento/%s/' %year
count = 0
for path in pathlib.Path(loc).iterdir():
    if path.is_file():
        count += 1

meses_load = []
for i in range(0, count-1):
    meses_load.append(meses[i])

print('Generando tablas… \nPor favor espere a ver el mensaje de que haya finalizado el proceso.')

for i in meses_load:
    nombre = ('%s_mo%s.csv' %(i,year))
    df = pd.read_csv(loc+nombre, sep=';') 
    df_gas = pd.DataFrame()

    limpieza(df,i, year)

print('Todas las tablas han sido creadas en el directorio: \n '
      './Tablas/year_%s/' %(year))


Introduce el año a estudiar: 
2020 -> 20
2019 -> 19


19


Tenga en cuenta que no todas las estaciones de medición tienen la capacidad de medir todos los gases.
Para más información, consultar: https://datos.madrid.es/portal/site/egob/ 


Generando tablas… 
Por favor espere a ver el mensaje de que haya finalizado el proceso.
Todas las tablas han sido creadas en el directorio: 
 ./Tablas/year_19/
